In [1]:
# importing packages

import pandas as pd
import numpy as np
import sys
sys.path.append("..")
from src.train import train_model
from src.score import score
from sklearn.metrics import f1_score
import mlflow

2024-08-04 17:13:22.783378: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-04 17:13:22.970680: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-04 17:13:23.149732: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-04 17:13:23.321483: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-04 17:13:23.366898: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-04 17:13:23.667775: I tensorflow/core/platform/cpu_feature_gu

In [2]:
# loading data, and separating it into inputs, and targets

df = pd.read_csv("../data/train.csv", sep=";")
target = df['FORCE_2020_LITHOFACIES_LITHOLOGY']
df = df.drop(['FORCE_2020_LITHOFACIES_LITHOLOGY', 'FORCE_2020_LITHOFACIES_CONFIDENCE'], axis=1)

: 

In [3]:
# calling the train function

train_model(df, target)

step 1: Data Transformation Complete!


2024/08/04 17:13:43 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!
/home/ubuntu/miniconda3/envs/oil/lib/python3.11/site-packages/_distutils_hack/__init__.py:26: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
2024/08/04 17:13:51 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.


step 2: Model Trained!


2024/08/04 17:29:56 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!


In [ ]:
# loading the test data and target

X_test = pd.read_csv("../data/test.csv", sep=";")
y_test = pd.read_csv("../data/leaderboard_test_target.csv", sep=";")['FORCE_2020_LITHOFACIES_LITHOLOGY']
X_test.head()

In [ ]:
# defining dictionary to map our predictions to, for our custom score function

lithology_keys = {30000: 'Sandstone',
                 65030: 'Sandstone/Shale',
                 65000: 'Shale',
                 80000: 'Marl',
                 74000: 'Dolomite',
                 70000: 'Limestone',
                 70032: 'Chalk',
                 88000: 'Halite',
                 86000: 'Anhydrite',
                 99000: 'Tuff',
                 90000: 'Coal',
                 93000: 'Basement'}

lithology_num = {'Sandstone': 0,
                 'Sandstone/Shale': 1,
                 'Shale': 2,
                 'Marl': 3,
                 'Dolomite': 4,
                 'Limestone': 5,
                 'Chalk': 6,
                 'Halite': 7,
                 'Anhydrite': 8,
                 'Tuff': 9,
                 'Coal': 10,
                 'Basement': 11}

y_test = y_test.map(lithology_keys)

In [ ]:
# loading the custom mlflow model

run_id = "3ec726621da64ec39cfddd20fc27a575"
custom_model = mlflow.pyfunc.load_model(f"runs:/{run_id}/lithology_classifier")

In [ ]:
# making predictions on the test set

y_pred = custom_model.predict(test)

#### Evaluation


In [ ]:
# evaluating model performance

y_pred_num = pd.Series(y_pred).map(lithology_num)
y_test_num = y_test.map(lithology_num)

score_ = score(y_test_num, y_pred_num)
f1score = f1_score(y_test, y_pred, average="weighted")

print(f"Custom score: {score_:.4f}")
print(f"f1-score: {f1score:.1%}")